In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/pollution.csv')

In [4]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'date'], infer_datetime_format=True)

In [6]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0,2010-01-02 00:00:00
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0,2010-01-02 01:00:00
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0,2010-01-02 02:00:00
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0,2010-01-02 03:00:00
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [7]:
df.columns

Index(['date', 'pollution', 'dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain',
       'NE', 'NW', 'SE', 'cv', 'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(35040, 13)

In [10]:
test_df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
35040,2014-01-01 00:00:00,24.0,-20,7.0,1014.0,143.48,0,0,0,1,0,0,2014-01-01 00:00:00
35041,2014-01-01 01:00:00,53.0,-20,7.0,1013.0,147.50,0,0,0,1,0,0,2014-01-01 01:00:00
35042,2014-01-01 02:00:00,65.0,-20,6.0,1013.0,151.52,0,0,0,1,0,0,2014-01-01 02:00:00
35043,2014-01-01 03:00:00,70.0,-20,6.0,1013.0,153.31,0,0,0,1,0,0,2014-01-01 03:00:00
35044,2014-01-01 04:00:00,79.0,-18,3.0,1012.0,0.89,0,0,0,0,0,1,2014-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [11]:
train_ds = PandasDataset(
    train_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

In [12]:
test_ds = PandasDataset(
    test_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

#### training

In [16]:
K = 6
n_hours = 24*3

In [17]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=200,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=2,
#     time_features=['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv'],
)

model = estimator.train(train_ds)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:30<00:00,  1.11it/s, epoch=76/200, avg_epoch_loss=6.41]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:30<00:00,  1.10it/s, epoch=152/200, avg_epoch_loss=6.46]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:31<00:00,  1.09it/s, epoch=200/200, avg_epoch_loss=6.39]


#### predicted y

In [18]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [19]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185]
(6,)
------
1
[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185  81.90731049  69.92803192  55.65983963  39.95516586
  25.57894897  15.70501995]
(12,)
------
2
[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185  81.90731049  69.92803192  55.65983963  39.95516586
  25.57894897  15.70501995  81.37012482  68.00048828  51.33604431
  33.72558594  21.34385872  14.17443943]
(18,)
------
3
[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185  81.90731049  69.92803192  55.65983963  39.95516586
  25.57894897  15.70501995  81.37012482  68.00048828  51.33604431
  33.72558594  21.34385872  14.17443943  84.29207611  73.32865143
  61.04417038  49.75625229  41.67584229  36.55028534]
(24,)
------
4
[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185  81.90731049  69.92803192  55.65983963  39.95516586
  25.5789

[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185  81.90731049  69.92803192  55.65983963  39.95516586
  25.57894897  15.70501995  81.37012482  68.00048828  51.33604431
  33.72558594  21.34385872  14.17443943  84.29207611  73.32865143
  61.04417038  49.75625229  41.67584229  36.55028534  67.32982635
  62.6637764   58.14544296  54.56965256  51.90363693  50.16144943
  76.46918488  76.43288422  75.99237061  75.47532654  75.04679108
  75.08507538  74.1122818   74.19315338  74.35530853  74.65947723
  75.02255249  75.24728394  80.35686493  80.09944916  79.28562164
  78.90350342  78.88141632  79.02935791  88.90920258  86.61404419
  84.85375977  84.58205414  85.0814743   86.27022552  96.27240753
  94.38748932  93.18959808  92.86922455  93.20424652  94.13201904
 103.17053986 101.87444305 100.86199188 100.44908905 100.89037323
 102.11940765 136.63818359 135.59501648 132.97311401 131.1688385
 132.30535889 135.91517639 159.25975037 159.33682251 158.98623657
 160.221511

[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185  81.90731049  69.92803192  55.65983963  39.95516586
  25.57894897  15.70501995  81.37012482  68.00048828  51.33604431
  33.72558594  21.34385872  14.17443943  84.29207611  73.32865143
  61.04417038  49.75625229  41.67584229  36.55028534  67.32982635
  62.6637764   58.14544296  54.56965256  51.90363693  50.16144943
  76.46918488  76.43288422  75.99237061  75.47532654  75.04679108
  75.08507538  74.1122818   74.19315338  74.35530853  74.65947723
  75.02255249  75.24728394  80.35686493  80.09944916  79.28562164
  78.90350342  78.88141632  79.02935791  88.90920258  86.61404419
  84.85375977  84.58205414  85.0814743   86.27022552  96.27240753
  94.38748932  93.18959808  92.86922455  93.20424652  94.13201904
 103.17053986 101.87444305 100.86199188 100.44908905 100.89037323
 102.11940765 136.63818359 135.59501648 132.97311401 131.1688385
 132.30535889 135.91517639 159.25975037 159.33682251 158.98623657
 160.221511

[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185  81.90731049  69.92803192  55.65983963  39.95516586
  25.57894897  15.70501995  81.37012482  68.00048828  51.33604431
  33.72558594  21.34385872  14.17443943  84.29207611  73.32865143
  61.04417038  49.75625229  41.67584229  36.55028534  67.32982635
  62.6637764   58.14544296  54.56965256  51.90363693  50.16144943
  76.46918488  76.43288422  75.99237061  75.47532654  75.04679108
  75.08507538  74.1122818   74.19315338  74.35530853  74.65947723
  75.02255249  75.24728394  80.35686493  80.09944916  79.28562164
  78.90350342  78.88141632  79.02935791  88.90920258  86.61404419
  84.85375977  84.58205414  85.0814743   86.27022552  96.27240753
  94.38748932  93.18959808  92.86922455  93.20424652  94.13201904
 103.17053986 101.87444305 100.86199188 100.44908905 100.89037323
 102.11940765 136.63818359 135.59501648 132.97311401 131.1688385
 132.30535889 135.91517639 159.25975037 159.33682251 158.98623657
 160.221511

[102.64434052  92.24625397  80.70789337  70.70348358  60.83961868
  50.16218185  81.90731049  69.92803192  55.65983963  39.95516586
  25.57894897  15.70501995  81.37012482  68.00048828  51.33604431
  33.72558594  21.34385872  14.17443943  84.29207611  73.32865143
  61.04417038  49.75625229  41.67584229  36.55028534  67.32982635
  62.6637764   58.14544296  54.56965256  51.90363693  50.16144943
  76.46918488  76.43288422  75.99237061  75.47532654  75.04679108
  75.08507538  74.1122818   74.19315338  74.35530853  74.65947723
  75.02255249  75.24728394  80.35686493  80.09944916  79.28562164
  78.90350342  78.88141632  79.02935791  88.90920258  86.61404419
  84.85375977  84.58205414  85.0814743   86.27022552  96.27240753
  94.38748932  93.18959808  92.86922455  93.20424652  94.13201904
 103.17053986 101.87444305 100.86199188 100.44908905 100.89037323
 102.11940765 136.63818359 135.59501648 132.97311401 131.1688385
 132.30535889 135.91517639 159.25975037 159.33682251 158.98623657
 160.221511

...

[102.64434052  92.24625397  80.70789337 ...  19.30101967  21.96259117
  24.16848183]
(49776,)
------
8296
[102.64434052  92.24625397  80.70789337 ...  13.63521576  14.69600296
  15.46509552]
(49782,)
------
8297
[102.64434052  92.24625397  80.70789337 ...  18.75496292  18.68832207
  18.6081562 ]
(49788,)
------
8298
[102.64434052  92.24625397  80.70789337 ...  19.03484726  19.39657402
  19.37332344]
(49794,)
------
8299
[102.64434052  92.24625397  80.70789337 ...  16.78617668  17.99736595
  16.1076889 ]
(49800,)
------
8300
[102.64434052  92.24625397  80.70789337 ...  12.56119061  12.05974865
  12.05668163]
(49806,)
------
8301
[102.64434052  92.24625397  80.70789337 ...  27.23427391  25.50422668
  21.86351013]
(49812,)
------
8302
[102.64434052  92.24625397  80.70789337 ...  12.31698322  11.31036758
  10.42227173]
(49818,)
------
8303
[102.64434052  92.24625397  80.70789337 ...  11.47029972  10.78612995
  10.19183254]
(49824,)
------
8304
[102.64434052  92.24625397  80.70789337 ...  1

[102.64434052  92.24625397  80.70789337 ... 248.54924011 239.43310547
 228.91778564]
(50256,)
------
8376
[102.64434052  92.24625397  80.70789337 ... 236.46324158 226.55986023
 216.12890625]
(50262,)
------
8377
[102.64434052  92.24625397  80.70789337 ... 262.20114136 253.34382629
 242.93452454]
(50268,)
------
8378
[102.64434052  92.24625397  80.70789337 ... 207.82714844 203.60322571
 201.04370117]
(50274,)
------
8379
[102.64434052  92.24625397  80.70789337 ... 217.28833008 212.54888916
 208.88798523]
(50280,)
------
8380
[102.64434052  92.24625397  80.70789337 ... 191.66343689 189.63787842
 188.64796448]
(50286,)
------
8381
[102.64434052  92.24625397  80.70789337 ... 172.81083679 169.54852295
 167.58361816]
(50292,)
------
8382
[102.64434052  92.24625397  80.70789337 ... 189.41064453 187.3177948
 186.17869568]
(50298,)
------
8383
[102.64434052  92.24625397  80.70789337 ... 204.61338806 205.09727478
 205.35466003]
(50304,)
------
8384
[102.64434052  92.24625397  80.70789337 ...  -2

[102.64434052  92.24625397  80.70789337 ... 123.05761719 122.69284821
 122.65476227]
(50748,)
------
8458
[102.64434052  92.24625397  80.70789337 ... 143.78941345 143.43218994
 144.0398407 ]
(50754,)
------
8459
[102.64434052  92.24625397  80.70789337 ... 132.26953125 133.2953949
 134.74829102]
(50760,)
------
8460
[102.64434052  92.24625397  80.70789337 ...  85.6623764   89.73062134
  95.38961029]
(50766,)
------
8461
[102.64434052  92.24625397  80.70789337 ...  82.57574463  90.35043335
 103.20510101]
(50772,)
------
8462
[102.64434052  92.24625397  80.70789337 ...  54.51398849  65.30557251
  78.61841583]
(50778,)
------
8463
[102.64434052  92.24625397  80.70789337 ... 124.84164429 156.30516052
 183.64494324]
(50784,)
------
8464
[102.64434052  92.24625397  80.70789337 ... 147.69697571 168.60289001
 189.71257019]
(50790,)
------
8465
[102.64434052  92.24625397  80.70789337 ...  99.16552734 107.71353912
 116.31919098]
(50796,)
------
8466
[102.64434052  92.24625397  80.70789337 ... 122

[102.64434052  92.24625397  80.70789337 ...  42.72000504  46.43136978
  46.74795151]
(51228,)
------
8538
[102.64434052  92.24625397  80.70789337 ...  58.03918457  59.40093994
  58.93484116]
(51234,)
------
8539
[102.64434052  92.24625397  80.70789337 ...  55.79356766  62.09100723
  58.79759598]
(51240,)
------
8540
[102.64434052  92.24625397  80.70789337 ...  63.68246078  63.42980957
  61.62929153]
(51246,)
------
8541
[102.64434052  92.24625397  80.70789337 ...  56.52120972  54.19465256
  50.18107986]
(51252,)
------
8542
[102.64434052  92.24625397  80.70789337 ...  51.12940598  48.14207458
  45.20228195]
(51258,)
------
8543
[102.64434052  92.24625397  80.70789337 ...  48.74610519  45.66794968
  41.55337906]
(51264,)
------
8544
[102.64434052  92.24625397  80.70789337 ...  62.25475693  56.4684639
  49.65407181]
(51270,)
------
8545
[102.64434052  92.24625397  80.70789337 ...  74.42848969  66.33487701
  58.30391312]
(51276,)
------
8546
[102.64434052  92.24625397  80.70789337 ...  63

[102.64434052  92.24625397  80.70789337 ... 128.34977722 128.08456421
 129.46711731]
(51738,)
------
8623
[102.64434052  92.24625397  80.70789337 ... 152.24499512 153.92591858
 156.37915039]
(51744,)
------
8624
[102.64434052  92.24625397  80.70789337 ... 123.63973999 125.99603271
 131.35713196]
(51750,)
------
8625
[102.64434052  92.24625397  80.70789337 ... 130.72564697 138.74237061
 149.19883728]
(51756,)
------
8626
[102.64434052  92.24625397  80.70789337 ... 169.85702515 177.29150391
 185.84510803]
(51762,)
------
8627
[102.64434052  92.24625397  80.70789337 ... 139.83370972 145.16282654
 154.15016174]
(51768,)
------
8628
[102.64434052  92.24625397  80.70789337 ... 150.78248596 159.96644592
 172.49589539]
(51774,)
------
8629
[102.64434052  92.24625397  80.70789337 ...  65.22559357  71.77697754
  79.2348175 ]
(51780,)
------
8630
[102.64434052  92.24625397  80.70789337 ...  73.56764984  91.72278595
 114.28317261]
(51786,)
------
8631
[102.64434052  92.24625397  80.70789337 ...  5

In [20]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(8683, 6)

In [21]:
test_ds_predict_q50_supervised[0:3]

array([[102.64434052,  92.24625397,  80.70789337,  70.70348358,
         60.83961868,  50.16218185],
       [ 81.90731049,  69.92803192,  55.65983963,  39.95516586,
         25.57894897,  15.70501995],
       [ 81.37012482,  68.00048828,  51.33604431,  33.72558594,
         21.34385872,  14.17443943]])

#### actual y

In [22]:
from helper import series_to_supervised, stage_series_to_supervised

In [23]:
test_df_supervised = series_to_supervised(test_df[['pollution']], n_hours, K)

In [24]:
test_df_supervised.iloc[:, -K:].shape

(8683, 6)

In [25]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [26]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5)
35112,85.0,86.0,89.0,77.0,77.0,75.0
35113,86.0,89.0,77.0,77.0,75.0,80.0
35114,89.0,77.0,77.0,75.0,80.0,86.0
35115,77.0,77.0,75.0,80.0,86.0,95.0
35116,77.0,75.0,80.0,86.0,95.0,101.0
...,...,...,...,...,...,...
43790,9.0,11.0,8.0,9.0,10.0,8.0
43791,11.0,8.0,9.0,10.0,8.0,10.0
43792,8.0,9.0,10.0,8.0,10.0,10.0
43793,9.0,10.0,8.0,10.0,10.0,8.0


#### performance on test set

In [27]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 26.545
RMSE = 46.436
